# Bidict_nGmaps

> Implementation based on bidict

In [1]:
from combinatorial.notebooks.combinatorial.custom_gmaps import nGmap
from combinatorial.notebooks.combinatorial.zoo import G2_SQUARE_BOUNDED
from my_gmaps import MyBidict#, my_Gmaps

from collections import defaultdict
from itertools import chain, product
import logging

In [2]:
class bidict_nGmap:

    def __init__(self, n):

        self.n = n

        self.darts = set()
        self.alpha = [MyBidict() for _ in range(self.n + 1)]
        self.marks = defaultdict(lambda: defaultdict(lambda: False))
        
        # self.marks[index][dart]

        self.taken_marks = {-1}
        self.lowest_unused_dart_index = 0

    @classmethod
    def from_string(cls, string):
        lines = string.strip().split("\n")
        lines = [[int(k) for k in l.split(" ") if k != ""] for l in lines]

        return cls.from_list_of_lists(lines)

    @classmethod
    def from_list_of_lists(cls, ll):
        n = len(ll) - 1
        d = len(ll[0])

        darts = set(ll[0])

        assert all(set(l) == darts for l in ll)

        my_nGmap = cls(n)
        my_nGmap.darts.update(darts)

        #print(my_nGmap.alpha) #are Mybidict objects

        for alpha, l in zip(my_nGmap.alpha, ll):
            for a, b in zip(sorted(darts), l):
                #alpha[a] = b
                alpha.add_item(a, b)    ##### Carmine 13.10.21
                

        my_nGmap.lowest_unused_dart_index = max(darts) + 1

        return my_nGmap

    @property
    def is_valid(self):
        """
        checks condition 2 and 3 from definition 1
        (condition 1 is always true as a computer probably has finite memory)
        """
        for dart in self.darts:
            for alpha in self.alpha:
                ##### Carmine 13.10.21 ore 22.21
                
                #if alpha[alpha[dart]] != dart:
                index = alpha.get_item(dart)[0]
                #print(f'index: {index}')
                item = alpha.get_item(index)[0]
                #print(f'item: {item}')
                if item != dart:
                    return False
            for i in range(0, self.n - 1):  # n-1 not included
                alpha1 = self.alpha[i]
                for j in range(i + 2, self.n + 1):  # n+1 again not included
                    alpha2 = self.alpha[j]
                    if alpha1.get_item(alpha2.get_item(alpha1.get_item(alpha2.get_item(dart)[0])[0])[0])[0] != dart:
                        return False
        return True

    def reserve_mark(self):
        """
        algorithm 2
        """
        i = max(self.taken_marks) + 1
        self.taken_marks.add(i)
        return i

    def free_mark(self, i):
        """
        algorithm 3
        also includes deleting all these marks, making it safe to call everytime
        """
        del self.marks[i]
        self.taken_marks.remove(i)

    def is_marked(self, d, i):
        """
        algorithm 4
        d ... dart
        i ... mark index
        """
        return self.marks[i][d]

    def mark(self, d, i):
        """
        algorithm 5
        d ... dart
        i ... mark index
        """
        self.marks[i][d] = True

    def unmark(self, d, i):
        """
        algorithm 6
        d ... dart
        i ... mark index
        """
        # same as bla = False
        del self.marks[i][d]

    def mark_all(self, i):
        for d in self.darts:
            self.mark(d, i)

    def unmark_all(self, i):
        del self.marks[i]

    def ai(self, i, d):
        return self.alpha[i][d]

    def set_ai(self, i, d, d1):
        assert 0 <= i <= self.n
        self.alpha[i][d] = d1

    def _remove_dart(self, d):
        self.darts.remove(d)
        for i in self.all_dimensions:
            del self.alpha[i][d]

    def orbit(self, sequence, d):
        """
        algorithm 7
        sequence ... valid list of dimensional indices
        d ... dart
        """
        ma = self.reserve_mark()
        self.mark_orbit(sequence, d, ma)
        orbit = self.marks[ma].keys()
        self.free_mark(ma)
        return orbit

    def mark_orbit(self, sequence, d, ma):
        """
        used as in algorithm 7 and 8 etc...
        sequence ... valid list of dimension indices
        d ... dart
        ma ... mark to use
        """
        P = [d]
        self.mark(d, ma)
        while P:
            cur = P.pop()
            for i in sequence:
                #other = self.alpha[i][cur]
                type_alpha = type(self.alpha[i])
                #print(f'type alpha: {type_alpha}')
                other = self.alpha[i].get_item(cur) ##### Carmine 13.10.21
                #print(f'other: {other[0]}')
                if not self.is_marked(other[0], ma):  ##### Carmine 13.10.21 ore 17:26
                    self.mark(other[0], ma) ##### Carmine 13.10.21 ore 17:30
                    P.append(other[0])  ##### Carmine 13.10.21 ore 17:30

    def cell_i(self, i, dart):
        """iterator over i-cell of a given dart"""
        return self.orbit(self.all_dimensions_but_i(i), dart)

    def cell_0(self, dart):
        return self.cell_i(0, dart)

    def cell_1(self, dart):
        return self.cell_i(1, dart)

    def cell_2(self, dart):
        return self.cell_i(2, dart)

    def cell_3(self, dart):
        return self.cell_i(3, dart)

    def cell_4(self, dart):
        return self.cell_i(4, dart)

    def no_i_cells(self, i=None):
        """
        Counts
            i-cells,             if 0 <= i <= n
            connected components if i is None
        """
        assert i is None or 0 <= i <= self.n
        # return more_itertools.ilen (self.darts_of_i_cells(i))
        return sum((1 for d in self.darts_of_i_cells(i)))

    @property
    def no_0_cells(self): return self.no_i_cells(0)
    @property
    def no_1_cells(self): return self.no_i_cells(1)
    @property
    def no_2_cells(self): return self.no_i_cells(2)
    @property
    def no_3_cells(self): return self.no_i_cells(3)
    @property
    def no_4_cells(self): return self.no_i_cells(4)
    @property
    def no_ccs(self): return self.no_i_cells()

    def darts_of_i_cells(self, i=None):
        """
        algorithm 8
        """
        ma = self.reserve_mark()
        try:
            for d in self.darts:
                if not self.is_marked(d, ma):
                    yield d
                    self.mark_orbit(self.all_dimensions_but_i(i), d, ma)
        finally:
            self.free_mark(ma)

    def all_i_cells(self, i=None):
        for d in self.darts_of_i_cells(i):
            yield self.cell_i(i, d)

    def all_conected_components(self):
        return self.all_i_cells()

    def darts_of_i_cells_incident_to_j_cell(self, d, i, j):
        """
        algorithm 9
        """
        assert i != j
        ma = self.reserve_mark()
        try:
            for e in self.orbit(self.all_dimensions_but_i(j), d):
                if not self.is_marked(e, ma):
                    yield e
                    self.mark_orbit(self.all_dimensions_but_i(j), e, ma)
        finally:
            self.free_mark(ma)

    def darts_of_i_cells_adjacent_to_i_cell(self, d, i):
        """
        algorithm 10
        """
        ma = self.reserve_mark()
        try:
            for e in self.orbit(self.all_dimensions_but_i(i), d):
                f = self.alpha[i].get_item(e)[0]    ##### Carmine 13.10.21 ore 17.44
                #print(f'f: {f}')
                if not self.is_marked(f, ma):
                    yield f
                    self.mark_orbit(self.all_dimensions_but_i(i), f, ma)
        finally:
            self.free_mark(ma)

    def all_dimensions_but_i(self, i=None):
        """Return a sorted sequence [0,...,n], without i, if 0 <= i <= n"""
        assert i is None or 0 <= i <= self.n
        return [j for j in range(self.n+1) if j != i]

    @property
    def all_dimensions(self):
        return self.all_dimensions_but_i()

    def is_i_free(self, i, d):
        """
        definiton 2 / algorithm 11
        """
        return self.alpha[i].get_item(d)[0] == d   ##### Carmine 13.10.21 ore 18.31

    def is_i_sewn_with(self, i, d):
        """
        definiton 2
        """
        d2 = self.alpha[i][d]
        return d != d2, d2

    def create_dart(self):
        """
        algorithm 12
        """
        d = self.lowest_unused_dart_index
        self.lowest_unused_dart_index += 1
        self.darts.add(d)
        for alpha in self.alpha:
            alpha.add_item(d, d) ##### Carmine 13.10.21 ore 17.57
        return d

    def remove_isolated_dart(self, d):
        """
        algorithm 13
        """
        assert self.is_isolated(d)
        self.remove_isolated_dart_no_assert(d)

    def remove_isolated_dart_no_assert(self, d):
        self.darts.remove(d)
        for alpha in self.alpha:
            #del alpha[d]
            alpha.delete_item(d)

    def is_isolated(self, d):
        for i in range(self.n + 1):
            if not self.is_i_free(i, d):
                return False
        return True

    ##### Carmine 13.10.21 ore 20.25
    def initialize(self):
        init_dict = dict()
        for d in self.darts:
            init_dict[d] = d

        return init_dict

    def increase_dim(self):
        """
        algorithm 15 in place
        """
        self.n += 1
        #self.alpha.append(dict((d, d) for d in self.darts))
        ##### Carmine 13.10.21 ore 20.25
        init_dict = self.initialize()
        self.alpha.append(MyBidict(init_dict))

        #print(self.alpha[3].get_normal_dict())        

    def decrease_dim(self):
        """
        algorithm 16 in place
        """
        assert all(self.is_i_free(self.n, d) for d in self.darts)
        self.decrease_dim_no_assert()

    def decrease_dim_no_assert(self):
        del self.alpha[self.n]
        self.n -= 1

    def index_shift(self, by):
        self.darts = {d + by for d in self.darts}
        self.alpha = [{k + by: v + by for k, v in a.get_normal_dict().items()}
                      for a in self.alpha]
        for mark in self.marks:
            new_dict = {key + by: value for key,
                        value in self.marks[mark].items()}
            self.marks[mark].clear()
            # this is done to preserve default dicts
            self.marks[mark].update(new_dict)
        self.lowest_unused_dart_index += by

    def merge(self, other):
        """
        algorithm 17 in place
        """
        print(f'n: {self.n} -> type: {type(self.n)}')
        print(f'\nother: {other.n} -> type: {type(other.n)}')
        assert self.n - 1 == other.n
        self.taken_marks.update(other.taken_marks)
        shift = max(self.darts) - min(other.darts) + 1
        other.index_shift(shift)

        self.darts.update(other.darts)
        for sa, oa in zip(self.alpha, other.alpha):
            sa.update(oa)
        for mk in other.marks:
            self.marks[mk].update(other.marks[mk])
        self.taken_marks.update(other.taken_marks)
        self.lowest_unused_dart_index = other.lowest_unused_dart_index

    def restrict(self, D):
        """
        algorithm 18
        """
        raise NotImplementedError  # boring

    def sew_seq(self, i):
        """
        indices used in the sewing operations
        (0, ..., i - 2, i + 2, ..., n)
        """
        return chain(range(0, i - 1), range(i + 2, self.n + 1))

    def sewable(self, d1, d2, i):
        """
        algorithm 19
        tests wether darts d1, d2 are sewable along i
        returns bool
        """
        if d1 == d2 or not self.is_i_free(i, d1) or not self.is_i_free(i, d2):
            return False
        try:
            f = dict()
            for d11, d22 in strict_zip(self.orbit(self.sew_seq(i), d1), self.orbit(self.sew_seq(i), d2), strict=True):
                f[d11] = d22
                for j in self.sew_seq(i):
                    if self.alpha[j][d11] in f and f[self.alpha[j][d11]] != self.alpha[j][d22]:
                        return False
        except ValueError:  # iterators not same length
            return False
        return True

    def sew(self, d1, d2, i):
        """
        algorithm 20
        """
        assert self.sewable(d1, d2, i)
        self.sew_no_assert(d1, d2, i)

    def sew_no_assert(self, d1, d2, i):
        for e1, e2 in strict_zip(self.orbit(self.sew_seq(i), d1), self.orbit(self.sew_seq(i), d2), strict=True):
            self.alpha[i][e1] = e2
            self.alpha[i][e2] = e1

    def unsew(self, d, i):
        """
        algorithm 21
        """
        assert not self.is_i_free(i, d)
        for e in self.orbit(self.sew_seq(i), d):
            f = self.alpha[i][e]
            self.alpha[i][f] = f
            self.alpha[i][e] = e

    def incident(self, i, d1, j, d2):
        """
        checks wether the i-cell of d1 is incident to the j-cell of d2
        """
        for e1, e2 in product(self.cell_i(i, d1), self.cell_i(j, d2)):
            if e1 == e2:
                return True
        return False

    def adjacent(self, i, d1, d2):
        """
        checks wether the i-cell of d1 is adjacent to the i-cell of d2
        """
        first_cell = self.cell_i(i, d1)
        second_cell = set(self.cell_i(i, d2))
        for d in first_cell:
            if self.alpha[i][d] in second_cell:
                return True
        return False

    # Contractablilty & Removability

    def _is_i_removable_or_contractible(self, i, dart, rc):
        """
        Test if an i-cell of dart is removable/contractible:

        i    ... i-cell
        dart ... dart
        rc   ... +1 => removable test, -1 => contractible test
        """
        assert dart in self.darts
        assert 0 <= i <= self.n
        assert rc in {-1, +1}

        if rc == +1:  # removable test
            if i == self.n:
                return False
            if i == self.n-1:
                return True
        if rc == -1:  # contractible test
            if i == 0:
                return False
            if i == 1:
                return True

        for d in self.cell_i(i, dart):
            if self.alpha[i+rc][self.alpha[i+rc+rc][d]] != self.alpha[i+rc+rc][self.alpha[i+rc][d]]:
                return False
        return True

    def is_i_removable(self, i, dart):
        """True if i-cell of dart can be removed"""
        return self._is_i_removable_or_contractible(i, dart, rc=+1)

    def is_i_contractible(self, i, dart):
        """True if i-cell of dart can be contracted"""
        return self._is_i_removable_or_contractible(i, dart, rc=-1)

    def _i_remove_contract(self, i, dart, rc, skip_check=False):
        """
        Remove / contract an i-cell of dart
        d  ... dart
        i  ... i-cell
        rc ... +1 => remove, -1 => contract
        skip_check ... set to True if you are sure you can remove / contract the i-cell
        """
        logging.debug(
            f'{"Remove" if rc == 1 else "Contract"} {i}-Cell of dart {dart}')

        if not skip_check:
            assert self._is_i_removable_or_contractible(i, dart, rc),\
                f'{i}-cell of dart {dart} is not {"removable" if rc == 1 else "contractible"}!'

        i_cell = set(self.cell_i(i, dart))  # mark all the darts in ci(d)
        logging.debug(f'\n{i}-cell to be removed {i_cell}')
        for d in i_cell:
            d1 = self.ai(i, d)  # d1 ← d.Alphas[i];
            if d1 not in i_cell:  # if not isMarkedNself(d1,ma) then
                # d2 ← d.Alphas[i + 1].Alphas[i];
                d2 = self.ai(i+rc, d)
                d2 = self.ai(i, d2)
                while d2 in i_cell:  # while isMarkedNself(d2,ma) do
                    # d2 ← d.Alphas[i + 1].Alphas[i];
                    d2 = self.ai(i+rc, d2)
                    d2 = self.ai(i, d2)
                logging.debug(
                    f'Modifying alpha_{i} of dart {d1} from {self.ai (i,d1)} to {d2}')

                self.set_ai(i, d1, d2)  # d1.Alphas[i] ← d2;
        for d in i_cell:  # foreach dart d' ∈ ci(d) do
            self._remove_dart(d)  # remove d' from gm.Darts;

    def _remove(self, i, dart, skip_check=False):
        """Remove i-cell of dart"""
        self._i_remove_contract(i, dart, rc=+1, skip_check=skip_check)

    def _contract(self, i, dart, skip_check=False):
        """Contract i-cell of dart"""
        self._i_remove_contract(i, dart, rc=-1, skip_check=skip_check)

    def __repr__(self):
        out = f"{self.n}dGmap of {len(self.darts)} darts:\n"
        for i in range(self.n + 1):
            out += f" {i}-cells: {self.no_i_cells(i)}\n"
        out += f" ccs: {self.no_ccs}"
        return out


def strict_zip(arg1, arg2, strict=False):
    """
    strict keyword for zip is only avaliable in python 3.10 which is still in alpha :(
    """
    assert strict == True
    arg1 = list(arg1)
    arg2 = list(arg2)
    if len(arg1) == len(arg2):
        return zip(arg1, arg2)
    else:
        raise ValueError


In [3]:
from combinatorial.notebooks.combinatorial.zoo import G2_HOUSE_1


m = bidict_nGmap.from_string(G2_HOUSE_1)

m

2dGmap of 24 darts:
 0-cells: 5
 1-cells: 6
 2-cells: 3
 ccs: 1

In [4]:
list(m.darts_of_i_cells(2))

[1, 9, 15]

In [5]:
list(m.darts_of_i_cells_incident_to_j_cell(1, 2, 1))

[1]

In [6]:
list(m.darts_of_i_cells_adjacent_to_i_cell(1, 2))

[15, 10]

In [7]:
d = m.create_dart()
d

25

In [8]:
m.remove_isolated_dart(d)

In [9]:
m.increase_dim()

In [10]:
m

3dGmap of 24 darts:
 0-cells: 5
 1-cells: 6
 2-cells: 3
 3-cells: 1
 ccs: 1

In [11]:
m.is_valid

True

In [12]:
m.merge(bidict_nGmap.from_string(G2_HOUSE_1))
m.is_valid

n: 3 -> type: <class 'int'>

other: 2 -> type: <class 'int'>


TypeError: can only concatenate list (not "int") to list

In [ ]:
m

3dGmap of 24 darts:
 0-cells: 5
 1-cells: 6
 2-cells: 3
 3-cells: 1
 ccs: 1

# END OF IMPLEMENTATION FROM DICT N-GMAP / START OF THE TOY EXAMPLE IMPLEMENTATION

In [ ]:
print (G2_SQUARE_BOUNDED)   #  this is just a multi-line string

# Asking user if wants to reconstruct the lower layer of the pyramid after removal/contraction operation

In [ ]:
def check_input(user_input):
    if user_input == '0' or user_input == '1':
        return True
    return False


In [ ]:
flag = False

while flag == False:
    user_input = input('Do you want to reconstruct the lower layer of the pyramid?\n Yes = 1\nNo = 0\n')
    flag = check_input(user_input)

print(flag)

In [ ]:
#G = nGmap.from_string(G2_SQUARE_BOUNDED)
G = my_Gmaps.from_string (G2_SQUARE_BOUNDED, flag)
G


In [ ]:
G.print_alpha_table()

# Implementation using bidict data structure

## Removal

## INPUT: set of dart to remove

In [ ]:
dart = input("Please enter the value of the dart you want to remove:\n")
 
print(f'You want to remove dart {dart}')

INPUT: i-cell to remove

In [ ]:
i = input("Please enter the i-cell you want to remove:\n")
 
print(f'You want to remove {i}-cell')

Execution of a remove methods using a custom name considering the i given in input

In [ ]:
getattr(G, f'remove_{i}_cell')(int(dart))

In [ ]:
print(G.get_dict(int(i)))

In [ ]:
G.get_all_dict()


In [ ]:
a = get_pair_from_normal(d, i)
print(a)

Use of the switch case instead of the if cases. #to improve

In [ ]:
def switch_func(i):
    print(i)
    
    switcher = {
        0: G.remove_0_cell(int(dart)),
        1: G.remove_1_cell(int(dart)),
        2: G.remove_2_cell(int(dart))
    }

    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(i, "Invalid value")

Snippet to have a list of methods of a specific class

In [ ]:
object_methods = [method_name for method_name in dir(my_Gmaps)
                  if callable(getattr(my_Gmaps, method_name))]

print(object_methods)
getattr(G, 'test_method')()

In [ ]:
import sys
s = 's'
print(sys.getsizeof(s))

In [ ]:
def byte_length(i):
    return (i.bit_length() + 7) // 8